Set of Data

In [ ]:
import pandas as pd
import numpy as np

# ============================
# 1. SET DATI
# ============================

train_cols = [c for c in train_df.columns if c not in ["battle_id", "player_won"]]
test_cols  = [c for c in test_df.columns if c != "battle_id"]

X = train_df[train_cols]
y = train_df["player_won"].astype(int)
X_test = test_df[test_cols]


NameError: name 'train_df' is not defined

# Models

Pipeline + Logistic

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

logit_clf = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(
            max_iter=2000,
            class_weight="balanced",
            solver="lbfgs",
            random_state=42
    ))
])


Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(
    n_estimators=1200,
    max_depth=20,
    max_features="sqrt",
    min_samples_leaf=2,
    min_samples_split=4,
    class_weight="balanced_subsample",
    n_jobs=-1,
    random_state=42
)

XGBoost

In [ ]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier(
    n_estimators=600,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.8,
    objective="binary:logistic",
    eval_metric="logloss",
    n_jobs=-1,
    random_state=42
)

Stacking Classifier [Ensemble]

In [ ]:
from sklearn.ensemble import StackingClassifier

stack_model = StackingClassifier(
    estimators=[
        ("logit", logit_clf),
        ("rf",    rf_clf),
        ("xgb",   xgb_clf)
    ],
    final_estimator=LogisticRegression(
        max_iter=2000,
        class_weight="balanced",
        random_state=42
    ),
    passthrough=False,
    n_jobs=-1
)

Cross-Validation

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("=== Cross-Validation Scores ===")

for name, model in [
    ("Logistic Regression", logit_clf),
    ("Random Forest", rf_clf),
    ("XGBoost", xgb_clf),
    ("Stack Ensemble", stack_model)
]:
    scores = cross_val_score(model, X, y, cv=cv, scoring="accuracy", n_jobs=-1)
    print(f"{name} | Mean ACC: {scores.mean():.4f} | STD: {scores.std():.4f}")



Final Training

In [ ]:
stack_model.fit(X, y)

Test Prediction & Submission

In [ ]:
test_pred = stack_model.predict(X_test)

submission = pd.DataFrame({
    "battle_id": raw_test_df["battle_id"],
    "player_won": test_pred.astype(int)
})

submission_path = "/kaggle/working/submission.csv"
submission.to_csv(submission_path, index=False)

print("Submission saved to:", submission_path)
submission.head()
